In [1]:
import time
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir=f"runs/mnist_training{int(time.time())}")

import torch.nn.functional as F

transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.2),
    transforms.RandomAffine(
        degrees=10,          # rotation légère
        translate=(0.1, 0.1),# petits décalages
        scale=(0.9, 1.1)     # zoom léger
    ),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NumberCNN(nn.Module):
    def __init__(self):
        super(NumberCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.2, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.2, training=self.training)   
        x = self.fc3(x)
        return x

model = NumberCNN().to(device)
print(model)

Using cpu device
NumberCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [5]:
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        step = epoch * len(dataloader) + batch
        writer.add_scalar("Loss/train", loss.item(), step)

        if batch % 100 == 0:
            loss_value, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")


In [7]:
accuracies = []

In [8]:
def test(dataloader, model, loss_fn, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    accuracy = correct / size

    writer.add_scalar("Accuracy/test", accuracy, epoch)

    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    accuracies.append(accuracy)

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, t)
    test(test_dataloader, model, loss_fn, t)
writer.close()
print("Done!")

Epoch 1
-------------------------------
loss: 2.297819  [   64/60000]
loss: 1.110272  [ 6464/60000]
loss: 0.889268  [12864/60000]
loss: 0.833503  [19264/60000]
loss: 0.670485  [25664/60000]
loss: 0.729526  [32064/60000]
loss: 0.725800  [38464/60000]
loss: 0.767556  [44864/60000]
loss: 0.805404  [51264/60000]
loss: 0.758030  [57664/60000]
Test Error: 
 Accuracy: 96.1%, Avg loss: 101.002086 

Epoch 2
-------------------------------
loss: 0.666241  [   64/60000]
loss: 0.682703  [ 6464/60000]
loss: 0.636461  [12864/60000]
loss: 0.684407  [19264/60000]
loss: 0.723644  [25664/60000]
loss: 0.648058  [32064/60000]
loss: 0.733075  [38464/60000]
loss: 0.672697  [44864/60000]
loss: 0.767419  [51264/60000]
loss: 0.701180  [57664/60000]
Test Error: 
 Accuracy: 96.8%, Avg loss: 96.146385 

Epoch 3
-------------------------------
loss: 0.638540  [   64/60000]
loss: 0.660268  [ 6464/60000]
loss: 0.676413  [12864/60000]
loss: 0.702726  [19264/60000]
loss: 0.660847  [25664/60000]
loss: 0.647691  [32064/

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NumberCNN().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [12]:
classes = [
    "Zero",
    "One",
    "Two",
    "Three",
    "Four",
    "Five",
    "Six",
    "Seven",
    "Eight",
    "Nine",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Seven", Actual: "Seven"


In [13]:
import statistics

mean_acc = statistics.mean(accuracies)
median_acc = statistics.median(accuracies)

print(f"Moyenne des accuracies test sur {epochs} époques : {mean_acc:.4f}")
print(f"Médiane des accuracies test : {median_acc:.4f}")

writer.add_scalar("Stats/mean_accuracy", mean_acc, epochs)
writer.add_scalar("Stats/median_accuracy", median_acc, epochs)
writer.close()
print("Done!")

Moyenne des accuracies test sur 5 époques : 0.9719
Médiane des accuracies test : 0.9758
Done!


In [14]:

%pip install onnx

dummy_input = torch.randn(1, 1, 28, 28, device=device)
torch.onnx.export(
    model, 
    dummy_input, 
    "mnist_cnn.onnx",
    input_names=["input"], 
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)
print("Modèle exporté en mnist_cnn.onnx")


You should consider upgrading via the 'c:\Users\Absam\Documents\IIM M1\ML-A4\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
Modèle exporté en mnist_cnn.onnx
